# Parameter Management

Once we have chosen an architecture and set our hyperparameters, we proceed to  the training loop, where our goal is to find parameters values that minimize our loss function. After training, we will need these parameters in order to make future predictions. Additionally, we will sometimes wish to extracct the parametres either to reuse them in some other context, to save our model to disk so that it may be executed in other software, or for examination in the hope of gaining scientific understanding. 

Although most of the time we are able to ignore the nitty-gritty details of how parameters are declared and manipulated, relying on deep learnng frameworkds to do the heavy lifting. However, when we move away from stacked architectures with standard layers, we will sometimes need to get into weeds of declaring and manipulating parameters. 

In [2]:
import torch 
from torch import nn
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(), nn.LazyLinear(1))

X = torch.rand(size=(2, 4))
net(X).shape

/Users/admin/miniconda3/envs/d2l/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 1])

## Parameters Access

When a model is defined vida the _Sequential_ class, we can first access any layer by indexing into the model as thoug it were a list. Each layer's parameters are conveniently located in its attribute. 

Inspecting the parameters of the second fully connected layers as follows:

In [3]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.1552, -0.0060, -0.3226, -0.0647,  0.2093, -0.2392,  0.0093,  0.1994]])),
             ('bias', tensor([0.1183]))])

In [4]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

## Targeted Parameters

In [5]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([0.1183]))

Each parameters is represented as an instance of the parameters class. To do anything useful with the parameters, we first need to access the underlying numerical values. The code above extracts the bias from the second neural network layer, which retursn a parameter class instance, and further accesses that parameter's value. 

## All Parameters at Once

In [7]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

## Tied Parameters

Often, we want to share parameters across multiple layers. Let's see how to do this elegantly. In the following we allocate a fully connected layer and then use its parameters specifically to set those of another layer. 

In [8]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
